In [1]:
import generator

In [2]:
from generator import util

In [3]:
import numpy as np

In [4]:
# define some colors (r,g,b)
yellow=[232, 181, 88]
l_blue=[94, 163, 188]
blue=[82, 110, 186]
orange=[186, 98, 82]

In [5]:
# generate test images
img_blue = np.tile([x / 255 for x in l_blue], (32, 64, 1))
img_yellow = np.tile([x / 255 for x in yellow], (32, 64, 1))

# load the two images
images = [
    generator.util.image.lin_to_srgb(img_blue),
    generator.util.image.lin_to_srgb(img_yellow)
]

In [6]:
# ------ create 'empty' Grid with num_rows, num_cols ---------
grid = generator.Grid(2, 3)
layout = grid.get_layout()
layout.set_padding(top=0.5, bottom=1.5)

# fill grid with image data
e1 = grid.get_element(0,0).set_image(images[0])
e2 = grid.get_element(0,1).set_image(images[1])
e3 = grid.get_element(0,2).set_image(images[0])
e4 = grid.get_element(1,0).set_image(images[1])
e5 = grid.get_element(1,1).set_image(images[0])
e6 = grid.get_element(1,2).set_image(images[1])

In [7]:
# marker (default marker props)
e1.set_marker(pos=[32,12], size=[15,10], color=blue)
e1.set_marker(pos=[1,1], size=[15,10], color=orange)

In [8]:
# frame
e2.set_frame(linewidth=2., color=blue)

In [9]:
# subtitles/captions for each element
e1.set_caption('hallo!')
e2.set_caption('und')
e3.set_caption('tschau!tschau!')
e4.set_caption('hallo!')
e5.set_caption('und')
e6.set_caption('tschau!tschau!')
layout.set_caption(height_mm=4.0, fontsize=9, txt_color=[170,170,170])

In [10]:
# labels (examples, each element can have in total 6 labels on each valid position)
e4.set_label("bottom center", pos='bottom_center', width_mm=25., height_mm=4.0, offset_mm=[1.0, 1.0], 
                  fontsize=9, bg_color=None)
e4.set_label("top\\\\right", pos='top_right', width_mm=8., height_mm=7.0, offset_mm=[1.0, 1.0], 
                  fontsize=9, bg_color=[255,255,255])
e4.set_label("top\\\\left", pos='top_left', width_mm=8., height_mm=7.0, offset_mm=[1.0, 1.0], 
                  fontsize=9, bg_color=orange, txt_color=[255,255,255])

In [11]:
# module titles
grid.set_title('top', 'Top Title')
layout.set_title('top', 5., offset_mm=2.,fontsize=12, bg_color=orange, txt_color=[255,255,255])

grid.set_title('left', 'Left Title')
layout.set_title('left', 4., offset_mm=2.,fontsize=12)

In [12]:
# Row and column titles
grid.set_row_titles('left', ['Row Titles', 'are better'])
layout.set_row_titles('left', 10., offset_mm=1., fontsize=9, txt_rotation=0, bg_color=[blue, l_blue])

grid.set_col_titles('north', ['Col Titles', 'are', 'The Best'])

In [13]:
generator.horizontal_figure([grid], width_cm=28., filename='singlemodule_test.pdf')
#generator.horizontal_figure([grid], width_cm=28., filename='singlemodule_test.pptx')
generator.horizontal_figure([grid], width_cm=28., filename='singlemodule_test.html')

total width of generated tikz module:  280.0
total height of generated tikz module:  107.93333333333332
Tikz/LaTeX file was generated.


In [14]:
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

In [24]:
PDF("singlemodule_test.pdf", (1000,500))